<a href="https://colab.research.google.com/github/xxoznge/Bigdata_Analyst_Certificate_Korean/blob/main/2%EC%9C%A0%ED%98%95_PRAC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# RandomForestClassifier : 분류 / 타겟 변수 : 0, 1, 2과 같은 이산형 데이터 : f1_score, accuracy 등
# RandomForestRegressor : 회귀 / 타겟 변수 : 150.0 과 같은 연속형 데이터 : mse, mae 등

In [67]:
# 모의고사 1회 2유형

# 데이터 불러오기

import pandas as pd

x_train = pd.read_csv('titanic3_X_train.csv')
x_test = pd.read_csv('titanic3_X_test.csv')
y_train = pd.read_csv('titanic3_y_train.csv')

# 정보 출력

print(x_train.head())
print(x_test.head())
print(y_train.head())

print(x_train.info())
print(x_test.info())
print(y_train.info())

print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

# 불필요 컬럼 제거

x_train = x_train.drop(['ID', 'name'], axis = 1)
x_test = x_test.drop(['ID', 'name'], axis = 1)

y_train = y_train.drop(['ID'], axis = 1)

# 결측치 제거

print(x_train.isna().sum())
print(x_test.isna().sum())

## 상관관계 계산해보기

cond_na = x_train['age'].isna()
from scipy.stats import pearsonr
pearsonr(y_train['survived'][~cond_na], x_train['age'][~cond_na])

x_train = x_train.drop('age', axis = 1)
x_test = x_test.drop('age', axis = 1)

## fare 가 train 쪽에서만 결측치 있으니 조건 달아서 행 삭제

cond_na = x_train['fare'].isna()
x_train = x_train[~cond_na]
y_train = y_train[~cond_na]

x_train = x_train.drop('cabin', axis = 1)
x_test = x_test.drop('cabin', axis = 1)

## embarked 탑승한 곳이니 최다빈도 범주로 대체

top = x_train['embarked'].value_counts().idxmax()

x_train['embarked'] = x_train['embarked'].fillna(top)
x_test['embarked'] = x_test['embarked'].fillna(top)

# 카테고리형 컬럼 전처리

print(x_train.select_dtypes('object').nunique())
print(x_test.select_dtypes('object').nunique())

## sex가 세 종류니까 확인

x_train['sex'].value_counts()

x_train['sex'] = x_train['sex'].map({'male':'male', 'female':'female', 'F':'female'})
x_test['sex'] = x_test['sex'].map({'male':'male', 'female':'female', 'F':'female'})

x_train = x_train.drop('ticket', axis = 1)
x_test = x_test.drop('ticket', axis = 1)

# 수치형 컬럼 전처리

## pclass 는 1, 2, 3으로 저장되지만 카테고리 의미 가짐

x_train['pclass_gp'] = x_train['pclass'].astype('object')
x_test['pclass_gp'] = x_test['pclass'].astype('object')

x_train = x_train.drop('pclass', axis = 1)
x_test = x_test.drop('pclass', axis = 1)

x_train['fam'] = x_train['parch'] + x_train['sibsp']
x_test['fam'] = x_test['parch'] + x_test['sibsp']

x_train = x_train.drop(['parch','sibsp'], axis = 1)
x_test = x_test.drop(['parch','sibsp'], axis = 1)

# 데이터 분할

from sklearn.model_selection import train_test_split

X_TRAIN, X_VAL, y_TRAIN, y_VAL = train_test_split(x_train, y_train, random_state = 1234, test_size = 0.1)

# 인코딩

from sklearn.preprocessing import OneHotEncoder

X_TRAIN_category = X_TRAIN.select_dtypes('object').copy()
X_VAL_category = X_VAL.select_dtypes('object').copy()
X_TEST_category = x_test.select_dtypes('object').copy()

enc = OneHotEncoder(sparse_output=False).fit(X_TRAIN_category)

X_TRAIN_OH = enc.transform(X_TRAIN_category)
X_VAL_OH = enc.transform(X_VAL_category)
X_TEST_OH = enc.transform(X_TEST_category)

# 스케일링

from sklearn.preprocessing import StandardScaler

X_TRAIN_conti = X_TRAIN.select_dtypes(exclude='object').copy()
X_VAL_conti = X_VAL.select_dtypes(exclude='object').copy()
X_TEST_conti = x_test.select_dtypes(exclude='object').copy()

scale = StandardScaler().fit(X_TRAIN_conti)

X_TRIAN_STD = scale.transform(X_TRAIN_conti)
X_VAL_STD = scale.transform(X_VAL_conti)
X_TEST_STD = scale.transform(X_TEST_conti)

# 입력 데이터셋 준비

import numpy as np

X_TRAIN = np.concatenate([X_TRAIN_OH, X_TRAIN_STD], axis = 1)
X_VAL = np.concatenate([X_VAL_OH, X_VAL_STD], axis = 1)

y_TRAIN = y_TRAIN.values.ravel()
y_VAL = y_VAL.values.ravel()

# 모델 학습

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 100,
                            max_depth = 3,
                            min_samples_leaf = 10,
                            max_features = 'sqrt',
                            random_state = 2022)

model_rf = rf.fit(X_TRAIN, y_TRAIN)

from sklearn.metrics import f1_score

pred_rf = model_rf.predict(X_VAL)

f1_rf = f1_score(y_VAL, pred_rf)
print(f1_rf)

# 결과 제출

X_TEST = np.concatenate([X_TEST_OH, X_TEST_STD], axis = 1)
y_pred = model_rf.predict(X_TEST)
obj = {'ID': ID,
       'survived':y_pred}
result = pd.DataFrame(obj)
result.to_csv('result.csv', index=False)


   ID  pclass      name   sex   age  sibsp  parch             ticket    fare  \
0   1       3  Sdy*****  male   NaN      0      0             349222  7.8958   
1   2       3  Pel*****  male  25.0      0      0  STON/O 2. 3101291  7.9250   
2   3       3  Kar*****  male  22.0      0      0             350060  7.5208   
3   4       3  Saa*****  male   NaN      0      0               2676  7.2250   
4   5       3  Cor*****  male  19.0      0      0             349231  7.8958   

  cabin embarked  
0   NaN        S  
1   NaN        S  
2   NaN        S  
3   NaN        C  
4   NaN        S  
    ID  pclass      name     sex   age  sibsp  parch    ticket      fare  \
0  786       1  All*****  female   2.0      1      2    113781  151.5500   
1  787       1  And*****    male  39.0      0      0    112050    0.0000   
2  788       1  Bau*****    male   NaN      0      0  PC 17318   25.9250   
3  789       1  Bax*****    male  24.0      0      1  PC 17558  247.5208   
4  790       1  Bea***** 

In [55]:
# 모의고사 2회 2유형

import pandas as pd

x_train = pd.read_csv("BlackFriday_X_train.csv")
x_test = pd.read_csv("BlackFriday_X_test.csv")
y_train = pd.read_csv("BlackFriday_y_train.csv")

print(x_train.head())
print(x_test.head())
print(y_train.head())

print(x_train.info())
print(x_test.info())
print(y_train.info())

print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

# 불필요 컬럼 제거

x_train = x_train.drop('Product_ID', axis = 1)
x_test = x_test.drop('Product_ID', axis = 1)

# 결측치 제거

print(x_train.isna().sum())
print(x_test.isna().sum())

x_train = x_train.drop('Product_Category_2', axis = 1)
x_test = x_test.drop('Product_Category_2', axis = 1)

x_train = x_train.drop('Product_Category_3', axis = 1)
x_test = x_test.drop('Product_Category_3', axis = 1)

# 카테고리형 컬럼

print(x_train.select_dtypes('object').nunique())
print(x_test.select_dtypes('object').nunique())

# 수치형 컬럼

print(x_train.select_dtypes(exclude = 'object'))
print(x_test.select_dtypes(exclude = 'object'))

x_train['Occ_gp'] = x_train['Occupation'].astype('object')
x_test['Occ_gp'] = x_test['Occupation'].astype('object')


x_train['Marital_gp'] = x_train['Marital_Status'].astype('object')
x_test['Marital_gp'] = x_test['Marital_Status'].astype('object')

x_train['PC_gp'] = x_train['Product_Category_1'].astype('object')
x_test['PC_gp'] = x_test['Product_Category_1'].astype('object')

x_train = x_train.drop(['Occupation', 'Marital_Status', 'Product_Category_1'], axis = 1)
x_test = x_test.drop(['Occupation', 'Marital_Status', 'Product_Category_1'], axis = 1)

# 데이터 분할

from sklearn.model_selection import train_test_split

X_TRAIN, X_VAL, y_TRAIN, y_VAL = train_test_split(x_train, y_train, random_state=1234, test_size = 0.3)

# 인코딩

from sklearn.preprocessing import OneHotEncoder

X_TRAIN_category = X_TRAIN.select_dtypes('object').copy()
X_VAL_category = X_VAL.select_dtypes('object').copy()
X_TEST_category = x_test.select_dtypes('object').copy()

enc = OneHotEncoder(sparse_output = False).fit(X_TRAIN_category)

X_TRAIN_OH = enc.transform(X_TRAIN_category)
X_VAL_OH = enc.transform(X_VAL_category)
X_TEST_OH = enc.transform(X_TEST_category)

# 스케일링 필요없음 ~conti

# 입력 데이터셋 준비

import numpy as np

X_TRAIN = X_TRAIN_OH # 만약 conti가 있다면 (+)
X_VAL = X_VAL_OH

# 평탄화 안함 .values.ravel() / 책에서는 하라고 했지만 에러 발생 !!!!

# 모델 학습

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100,
                           max_depth = 3,
                           min_samples_leaf = 10,
                           max_features = 2,
                           random_state = 2022)

model_rf = rf.fit(X_TRAIN, y_TRAIN)

from sklearn.metrics import mean_absolute_error

pred_rf = rf.predict(X_VAL)
mae_rf = mean_absolute_error(y_VAL, pred_rf)
print(mae_rf)

# 결과 제출

X_TEST = X_TEST_OH
y_pred = model_rf.predict(X_TEST).flatten() # 1차원 배열로 변환

obj = {'Purchase' : y_pred}
result = pd.DataFrame(obj)

result.to_csv('result.csv', index=False)

   User_ID Product_ID Gender    Age  Occupation City_Category  \
0  1001889  P00166642      M  18-25          14             B   
1  1003320  P00030842      M  26-35           1             B   
2  1003690   P0095742      M  18-25           0             B   
3  1002796  P00227642      M  26-35          14             B   
4  1001671  P00114542      M  36-45           0             B   

  Stay_In_Current_City_Years  Marital_Status  Product_Category_1  \
0                          2               0                   8   
1                          1               1                   1   
2                          2               0                   4   
3                          3               0                   1   
4                          2               0                   5   

   Product_Category_2  Product_Category_3  
0                 NaN                 NaN  
1                 2.0                15.0  
2                 5.0                 NaN  
3                 5.0   

In [ ]:
# 모의고사 3회 2유형

import pandas as pd

train = pd.read_csv("train_insurance.csv")
test = pd.read_csv("test_insurance.csv")

y_train = train['charges']
x_train = train.drop('charges', axis = 1)
x_test = test

print(x_train.head())
print(y_train.head())
print(x_test.head())

print(x_train.info())
print(y_train.info())
print(x_test.info())

print(x_train.describe())
print(y_train.describe())
print(x_test.describe())

print(x_train.isna().sum())
print(y_train.isna().sum())
print(x_test.isna().sum())

from sklearn.model_selection import train_test_split

X_TRAIN, X_VAL, y_TRAIN, y_VAL = train_test_split(x_train, y_train, random_state = 1234, test_size = 0.1)

from sklearn.preprocessing import OneHotEncoder

X_TRAIN_category = X_TRAIN.select_dtypes('object').copy()
X_VAL_category = X_VAL.select_dtypes('object').copy()
X_TEST_category = x_test.select_dtypes('object').copy()

enc = OneHotEncoder(sparse_output = False).fit(X_TRAIN_category)
X_TRAIN_OH = enc.transform(X_TRAIN_category)
X_VAL_OH = enc.transform(X_VAL_category)
X_TEST_OH = enc.transform(X_TEST_category)

X_TRAIN_conti = X_TRAIN.select_dtypes(exclude='object').copy()
X_VAL_conti = X_VAL.select_dtypes(exclude='object').copy()
X_TEST_conti = x_test.select_dtypes(exclude='object').copy()

import numpy as np

X_TRAIN = np.concatenate([X_TRAIN_OH, X_TRAIN_conti], axis = 1)
X_VAL = np.concatenate([X_VAL_OH, X_VAL_conti], axis = 1)

y_TRAIN = y_TRAIN.values.ravel()
y_VAL = y_VAL.values.ravel()

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100,
                            max_depth = 3,
                            min_samples_leaf = 10,
                            max_features = 15,
                            random_state = 2023)
model_rf = rf.fit(X_TRAIN, y_TRAIN)

from sklearn.metrics import mean_absolute_percentage_error

pred_rf = model_rf.predict(X_VAL)

mape_rf = mean_absolute_percentage_error(y_VAL, pred_rf)
print(mape_rf)

X_TEST = np.concatenate([X_TEST_OH, X_TEST_conti], axis = 1)
y_pred = model_rf.predict(X_TEST)

obj = {'pred':y_pred}
result = pd.DataFrame(obj)

result.to_csv("result.csv", index = False)


   age     sex     bmi  children smoker     region
0  10S  female  27.900         0    yes  southwest
1  10S    male  33.770         1     no  southeast
2  20S    male  33.000         3     no  southeast
3  30S    male  22.705         0     no  northwest
4  30S    male  28.880         0     no  northwest
0    16884.92400
1     1725.55230
2     4449.46200
3    21984.47061
4     3866.85520
Name: charges, dtype: float64
   age     sex    bmi  children smoker     region
0  20S  female  39.49         0     no  southeast
1  30S    male  33.82         1     no  northwest
2  20S    male  32.01         0     no  southeast
3  50S    male  27.94         1     no  southeast
4  50S    male  41.14         1    yes  southeast
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058 entries, 0 to 1057
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1058 non-null   object 
 1   sex       1058 non-null   object 
 2   bmi       10

In [83]:
# 기출복원 2회 2유형

# 데이터 불러오기

import pandas as pd

x_train = pd.read_csv('stroke_X_train.csv')
x_test = pd.read_csv('stroke_X_test.csv')
y_train = pd.read_csv('stroke_y_train.csv')

# 정보 출력

print(x_train.head())
print(x_test.head())
print(y_train.head())

print(x_train.info())
print(x_test.info())
print(y_train.info())

print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

# 불필요한 컬럼 제거

ID = x_test['id'].copy()

x_train = x_train.drop('id', axis = 1)
x_test = x_test.drop('id', axis = 1)
y_train = y_train.drop('id', axis = 1)

# 결측치 제거

print(x_train.isna().sum())
print(x_test.isna().sum())

x_train['smoking_status'].value_counts()

x_train['smoking_status'] = x_train['smoking_status'].fillna('unknown')
x_test['smoking_status'] = x_test['smoking_status'].fillna('unknown')

# 카테고리형 컬럼 안해도 됨

# 수치형 컬럼

## age 를 범주형으로

x_train['age'] = pd.cut(x_train['age'], bins=list(range(0, 100, 10))).astype('object')
x_test['age'] = pd.cut(x_test['age'], bins=list(range(0, 100, 10))).astype('object')

# 데이터 분할

from sklearn.model_selection import train_test_split

X_TRAIN, X_VAL, y_TRAIN, y_VAL = train_test_split(x_train, y_train, random_state=1234, test_size=0.2)

# 인코딩

from sklearn.preprocessing import OneHotEncoder

X_TRAIN_category = X_TRAIN.select_dtypes('object').copy()
X_VAL_category = X_VAL.select_dtypes('object').copy()
X_TEST_category = x_test.select_dtypes('object').copy()

enc = OneHotEncoder(sparse_output=False).fit(X_TRAIN_category)

X_TRAIN_OH = enc.transform(X_TRAIN_category)
X_VAL_OH = enc.transform(X_VAL_category)
X_TEST_OH = enc.transform(X_TEST_category)

# 스케일링

from sklearn.preprocessing import StandardScaler

X_TRAIN_conti = X_TRAIN.select_dtypes(exclude='object').copy()
X_VAL_conti = X_VAL.select_dtypes(exclude='object').copy()
X_TEST_conti = x_test.select_dtypes(exclude='object').copy()

scale = StandardScaler().fit(X_TRAIN_conti)

X_TRAIN_STD = scale.transform(X_TRAIN_conti)
X_VAL_STD = scale.transform(X_VAL_conti)
X_TEST_STD = scale.transform(X_TEST_conti)

# 입력데이터셋 준비

import numpy as np

X_TRAIN = np.concatenate([X_TRAIN_OH, X_TRAIN_STD], axis = 1)
X_VAL = np.concatenate([X_VAL_OH, X_VAL_STD], axis = 1)

y_TRAIN = y_TRAIN.values.ravel()
y_VAL = y_VAL.values.ravel()

# 모델 학습

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 100,
                            max_depth = 3,
                            min_samples_leaf = 10,
                            max_features = 2,
                            random_state = 2022)
model_rf = rf.fit(X_TRAIN, y_TRAIN)

# 성능 검증

from sklearn.metrics import accuracy_score

pred_rf = model_rf.predict(X_VAL)

acc_rf = accuracy_score(y_VAL, pred_rf)
print(acc_rf)

# 결과 제출

X_TEST = np.concatenate([X_TEST_OH, X_TEST_STD], axis = 1)
y_pred = model_rf.predict(X_TEST)

obj = {'id':ID,
       'stroke':y_pred}
result = pd.DataFrame(obj)

result.to_csv('result.csv', index=False)

     id  gender   age  hypertension  heart_disease ever_married work_type  \
0   678    Male   8.0             0              0           No  children   
1  1126  Female  65.0             0              0          Yes  Govt_job   
2  1125  Female  79.0             1              0          Yes   Private   
3  1016    Male  66.0             0              0          Yes   Private   
4   948  Female  45.0             0              0          Yes   Private   

  Residence_type  avg_glucose_level   bmi   smoking_status  
0          Urban              78.48  16.1          Unknown  
1          Urban              82.72  29.8           smokes  
2          Rural             119.62  39.0          Unknown  
3          Rural              67.92  31.1  formerly smoked  
4          Urban             146.44  22.8  formerly smoked  
   id  gender   age  hypertension  heart_disease ever_married work_type  \
0   2  Female  78.0             0              0          Yes   Private   
1   4  Female  59.0  

In [4]:
# 기출복원 3회 2유형

# 데이터 불러오기

import pandas as pd

x_train = pd.read_csv('job_change_X_train.csv')
x_test = pd.read_csv('job_change_X_test.csv')
y_train = pd.read_csv('job_change_y_train.csv')

# 정보 출력

print(x_train.head())
print(x_test.head())
print(y_train.head())

print(x_train.info())
print(x_test.info())
print(y_train.info())

print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

# 불필요한 컬럼 제거

enrollee_id = x_test['enrollee_id'].copy()

x_train = x_train.drop('enrollee_id', axis = 1)
x_test = x_test.drop('enrollee_id', axis = 1)
y_train = y_train.drop('enrollee_id', axis = 1)

# 결측치 제거

print(x_train.isna().sum())
print(x_test.isna().sum())

## 1000개 이상은 없애버리기

cond_na1000 = (x_train.isna().sum() > 1000)

colnm_na1000 = x_train.columns[cond_na1000]

x_train = x_train.drop(colnm_na1000, axis = 1)
x_test = x_test.drop(colnm_na1000, axis = 1)

## 200개 미만은 최다 빈도로 대체

mode_EU = x_train['enrolled_university'].value_counts().idxmax()

x_train['enrolled_university'] = x_train['enrolled_university'].fillna(mode_EU)
x_test['enrolled_university'] = x_test['enrolled_university'].fillna(mode_EU)

mode_EL = x_train['education_level'].value_counts().idxmax()

x_train['education_level'] = x_train['education_level'].fillna(mode_EL)
x_test['education_level'] = x_test['education_level'].fillna(mode_EL)

# 카테고리형 컬럼 없음

# 수치형 컬럼 없음

# 데이터 분할

from sklearn.model_selection import train_test_split

X_TRAIN, X_VAL, y_TRAIN, y_VAL = train_test_split(x_train, y_train, random_state=1234, test_size=0.2)

# 인코딩

from sklearn.preprocessing import OneHotEncoder

X_TRAIN_category = X_TRAIN.select_dtypes('object').copy()
X_VAL_category = X_VAL.select_dtypes('object').copy()
X_TEST_category = x_test.select_dtypes('object').copy()

enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore').fit(X_TRAIN_category)

X_TRAIN_OH = enc.transform(X_TRAIN_category)
X_VAL_OH = enc.transform(X_VAL_category)
X_TEST_OH = enc.transform(X_TEST_category)

# 스케일링

from sklearn.preprocessing import StandardScaler

X_TRAIN_conti = X_TRAIN.select_dtypes(exclude='object').copy()
X_VAL_conti = X_VAL.select_dtypes(exclude='object').copy()
X_TEST_conti = x_test.select_dtypes(exclude='object').copy()

scale = StandardScaler().fit(X_TRAIN_conti)

X_TRAIN_STD = scale.transform(X_TRAIN_conti)
X_VAL_STD = scale.transform(X_VAL_conti)
X_TEST_STD = scale.transform(X_TEST_conti)

# 입력데이터셋 준비

import numpy as np

X_TRAIN = np.concatenate([X_TRAIN_OH, X_TRAIN_STD], axis = 1)
X_VAL = np.concatenate([X_VAL_OH, X_VAL_STD], axis = 1)

y_TRAIN = y_TRAIN.values.ravel()
y_VAL = y_VAL.values.ravel()

# 모델 학습

## RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 500,
                            max_depth = 3,
                            min_samples_leaf = 10,
                            max_features = 'sqrt',
                            random_state = 2022)
model_rf = rf.fit(X_TRAIN, y_TRAIN)

## XGBoost
from xgboost import XGBClassifier

xgb = XGBClassifier(max_depth = 8,
                    n_estimators = 500,
                    nthread = 5,
                    min_child_weight = 20,
                    gamma = 0.5,
                    objective = 'binary:logistic',
                    use_label_encoder = False,
                    random_state = 2022)

model_xgb = xgb.fit(X_TRAIN, y_TRAIN)

# 성능 검정

from sklearn.metrics import roc_curve, auc

score_rf = model_rf.predict_proba(X_VAL)[:,1]

fpr, tpr, thresholds = roc_curve(y_VAL, score_rf)
auc_rf = auc(fpr, tpr)
print(auc_rf)

score_xgb = model_xgb.predict_proba(X_VAL)[:,1]

fpr, tpr, thresholds = roc_curve(y_VAL, score_xgb)
auc_xgb = auc(fpr, tpr)
print(auc_xgb)

# 결과 제출

X_TEST = np.concatenate([X_TEST_OH, X_TEST_STD], axis = 1)
y_pred = model_rf.predict(X_TEST)
y_score = model_rf.predict_proba(X_TEST)[:,1]

obj = {'enrollee_id':enrollee_id,
       'target':y_pred,
       'target_prob':y_score}
result = pd.DataFrame(obj)

result.to_csv('result.csv', index=False)

   enrollee_id      city  city_development_index gender relevent_experience  \
0         6424   city_75                   0.939   Male                 Has   
1        18304   city_64                   0.666   Male                  No   
2        22515  city_103                   0.920   Male                  No   
3        15878   city_16                   0.910   Male                 Has   
4        22128  city_103                   0.920   Male                 Has   

  enrolled_university education_level major_discipline experience  \
0                  No        Graduate             STEM         10   
1                Full            High              NaN          2   
2                Full            High              NaN          4   
3                  No        Graduate             STEM          8   
4                  No        Graduate             STEM          6   

  company_size         company_type last_new_job  training_hours  
0        50-99              Pvt Ltd        

In [16]:
# 기출복원 4회 2유형

# 데이터 불러오기

import pandas as pd

train = pd.read_csv('bodyPerfor_train.csv')
test = pd.read_csv('bodyPerfor_test.csv')

y_train = train['class'].copy()
x_train = train.drop(columns = 'class').copy()

x_test = test.copy()

# 정보 출력

print(x_train.head())
print(x_test.head())
print(y_train.head())

print(x_train.info())
print(x_test.info())
print(y_train.info())

print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

# 불필요한 컬럼 제거

x_train = x_train.drop('id', axis = 1)
x_test = x_test.drop('id', axis = 1)

# 결측치 제거

print(x_train.isna().sum())
print(x_test.isna().sum())

# 카테고리형 컬럼 전처리 생략

# 수치형 컬럼 전처리

## age 범주형으로

x_train['age'] = pd.cut(x_train['age'], bins = list(range(0, 100, 10))).astype('object')
x_test['age'] = pd.cut(x_test['age'], bins = list(range(0, 100, 10))).astype('object')

# 데이터 분할

from sklearn.model_selection import train_test_split

X_TRAIN, X_VAL, y_TRAIN, y_VAL = train_test_split(x_train, y_train, random_state = 1234, test_size=0.2)

# 인코딩

from sklearn.preprocessing import OneHotEncoder

X_TRAIN_category = X_TRAIN.select_dtypes('object').copy()
X_VAL_category = X_VAL.select_dtypes('object').copy()
X_TEST_category = x_test.select_dtypes('object').copy()

enc = OneHotEncoder(sparse_output=False).fit(X_TRAIN_category)

X_TRAIN_OH = enc.transform(X_TRAIN_category)
X_VAL_OH = enc.transform(X_VAL_category)
X_TEST_OH = enc.transform(X_TEST_category)

# 스케일링

from sklearn.preprocessing import StandardScaler

X_TRAIN_conti = X_TRAIN.select_dtypes(exclude='object').copy()
X_VAL_conti = X_VAL.select_dtypes(exclude='object').copy()
X_TEST_conti = x_test.select_dtypes(exclude='object').copy()

scale = StandardScaler().fit(X_TRAIN_conti)

X_TRAIN_STD = scale.transform(X_TRAIN_conti)
X_VAL_STD = scale.transform(X_VAL_conti)
X_TEST_STD = scale.transform(X_TEST_conti)

# 입력 데이터셋 준비

import numpy as np

X_TRAIN = np.concatenate([X_TRAIN_OH, X_TRAIN_STD], axis = 1)
X_VAL = np.concatenate([X_VAL_OH, X_VAL_STD], axis = 1)

y_TRAIN = y_TRAIN.map({'A':0, 'B':1, 'C':2, 'D':3})
y_VAL = y_VAL.map({'A':0, 'B':1, 'C':2, 'D':3})

y_TRAIN = y_TRAIN.values.ravel()
y_VAL = y_VAL.values.ravel()

# 모델 학습

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

rf = RandomForestClassifier(n_estimators = 500,
                            max_depth = 3,
                            min_samples_leaf = 10,
                            max_features = 'sqrt',
                            random_state = 2022)
model_rf = rf.fit(X_TRAIN, y_TRAIN)

xgb = XGBClassifier(max_depth = 8,
                    n_estimators = 500,
                    nthread = 5,
                    min_child_weight = 20,
                    objective = 'multi:softmax',
                    random_state = 2022)
model_xgb = xgb.fit(X_TRAIN, y_TRAIN)

# 성능 검정

from sklearn.metrics import f1_score

pred_rf = model_rf.predict(X_VAL)
pred_xgb = model_xgb.predict(X_VAL)

f1_rf = f1_score(y_VAL, pred_rf, average = 'macro')
print(f1_rf)

f1_xgb = f1_score(y_VAL, pred_xgb, average = 'macro')
print(f1_xgb)

# 결과 제출

X_TEST = np.concatenate([X_TEST_OH, X_TEST_STD], axis = 1)
y_pred = model_xgb.predict(X_TEST)

obj = {'class':y_pred}
result = pd.DataFrame(obj)

result.to_csv('result.csv', index = False)

      id  age gender  height  weight  body_fat  diastolic  systolic  \
0    920   40      M   173.8    73.2      12.0       80.0       148   
1   5998   35      M   176.2    73.7      15.7       64.0       119   
2  11457   57      M   169.2    73.5      21.7       77.0       128   
3   2898   45      M   170.3    76.7      17.0       86.0       154   
4   3141   28      M   171.3    68.7      11.2       99.0       145   

   grip_force  sit_bend_forward  sit_ups  broad_jump  
0        46.9              15.0     50.0       250.0  
1        47.6              18.0     46.0       226.0  
2        49.5              14.8     39.0       206.0  
3        50.8              19.5     43.0       214.0  
4        42.7              23.2     71.0       269.0  
   id  age gender  height  weight  body_fat  diastolic  systolic  grip_force  \
0  11   42      M   169.2    65.4      19.3       63.0     110.0        43.5   
1  22   59      F   155.9    62.7      30.2       76.0     143.0        36.8   
2  

In [31]:
# 기출복원 5회 2유형

# 데이터 불러오기

import pandas as np

train = pd.read_csv('carprice_train.csv')
test = pd.read_csv('carprice_test.csv')

y_train = train['price'].copy()
x_train = train.drop(columns = 'price').copy()

x_test = test.copy()

# 정보 출력

print(x_train.head())
print(x_test.head())
print(y_train.head())

print(x_train.info())
print(x_test.info())
print(y_train.info())

print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

# 불필요한 컬럼 제거

# 결측치 제거

print(x_train.isna().sum())
print(x_test.isna().sum())

# 카테고리형 컬럼 전처리

# 수치형 컬럼 전처리

x_train['year'] = x_train['year'].astype('object')
x_test['year'] = x_test['year'].astype('object')

# 데이터 분할

from sklearn.model_selection import train_test_split

X_TRAIN, X_VAL, y_TRAIN, y_VAL = train_test_split(x_train, y_train, random_state = 1234, test_size=0.2)

# 인코딩

from sklearn.preprocessing import OneHotEncoder

X_TRAIN_category = X_TRAIN.select_dtypes('object').copy()
X_VAL_category = X_VAL.select_dtypes('object').copy()
X_TEST_category = x_test.select_dtypes('object').copy()

enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore').fit(X_TRAIN_category)

X_TRAIN_OH = enc.transform(X_TRAIN_category)
X_VAL_OH = enc.transform(X_VAL_category)
X_TEST_OH = enc.transform(X_TEST_category)

# 스케일링

from sklearn.preprocessing import StandardScaler

X_TRAIN_conti = X_TRAIN.select_dtypes(exclude='object').copy()
X_VAL_conti = X_VAL.select_dtypes(exclude='object').copy()
X_TEST_conti = x_test.select_dtypes(exclude='object').copy()

scale = StandardScaler().fit(X_TRAIN_conti)

X_TRAIN_STD = scale.transform(X_TRAIN_conti)
X_VAL_STD = scale.transform(X_VAL_conti)
X_TEST_STD = scale.transform(X_TEST_conti)

# 입력 데이터셋 준비

import numpy as np

X_TRAIN = np.concatenate([X_TRAIN_OH, X_TRAIN_STD], axis = 1)
X_VAL = np.concatenate([X_VAL_OH, X_VAL_STD], axis = 1)

y_TRAIN = y_TRAIN.values.ravel()
y_VAL = y_VAL.values.ravel()

# 모델 학습

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

rf = RandomForestRegressor(n_estimators = 100,
                           max_depth = 3,
                           min_samples_leaf = 10,
                           max_features = 30,
                           random_state = 2023)
model_rf = rf.fit(X_TRAIN, y_TRAIN)

xgb = XGBRegressor(n_estimators = 100,
                   nthread = 5,
                   min_child_weight = 20,
                   gamma = 0.5,
                   objective = 'reg:squarederror',
                   use_label_encoder = False,
                   random_state = 2023)
model_xgb = xgb.fit(X_TRAIN, y_TRAIN)

# 성능 검정

from sklearn.metrics import mean_squared_error

pred_rf = model_rf.predict(X_VAL)
pred_xgb = model_xgb.predict(X_VAL)

rmse_rf = mean_squared_error(y_VAL, pred_rf, squared=False)
print(rmse_rf)

rmse_xgb = mean_squared_error(y_VAL, pred_xgb, squared=False)
print(rmse_xgb)

# 결과 제출

X_TEST = np.concatenate([X_TEST_OH, X_TEST_STD], axis = 1)
y_pred = model_rf.predict(X_TEST)

obj = {'pred':y_pred}
result = pd.DataFrame(obj)

result.to_csv('result.csv', index = False)


    model  year transmission  mileage fuelType  tax   mpg  engineSize
0    Kuga  2017       Manual    12109   Petrol  165  45.6         1.5
1    Kuga  2018       Manual    12125   Diesel  145  54.3         1.5
2    Kuga  2016       Manual    31288   Diesel  125  60.1         2.0
3  Fiesta  2018       Manual    28528   Petrol  145  65.7         1.0
4  Fiesta  2017       Manual     2690   Petrol  145  65.7         1.0
    model  year transmission  mileage fuelType  tax   mpg  engineSize
0    Kuga  2019    Automatic     6894   Diesel  145  42.2         2.0
1    Kuga  2018    Semi-Auto     9002   Diesel  145  54.3         2.0
2    Kuga  2014       Manual    59000   Diesel  160  47.9         2.0
3  Fiesta  2017       Manual    25000   Petrol  125  54.3         1.2
4  Fiesta  2017       Manual    15612   Petrol  145  65.7         1.0
0    14450
1    17500
2    14800
3    11000
4    11330
Name: price, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13470 entries, 0 to 13469
Dat

In [45]:
# 기출복원 6회 2유형

# 데이터 불러오기

import pandas as pd

train = pd.read_csv('Airline_train.csv')
test = pd.read_csv('Airline_test.csv')

y_train = train['Satisfaction'].copy()
x_train = train.drop(columns = 'Satisfaction').copy()

x_test = test.copy()

# 정보 출력

print(x_train.head())
print(x_test.head())
print(y_train.head())

print(x_train.info())
print(x_test.info())
print(y_train.info())

print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

# 불필요한 컬럼 제거

# 결측치 제거

print(x_train.isna().sum())
print(x_test.isna().sum())

# 카테고리형 컬럼 전처리

# 수치형 컬럼 전처리

# 데이터 분할

from sklearn.model_selection import train_test_split

X_TRAIN, X_VAL, y_TRAIN, y_VAL = train_test_split(x_train, y_train, random_state = 1234, test_size = 0.2)

# 인코딩

from sklearn.preprocessing import OneHotEncoder

X_TRAIN_category = X_TRAIN.select_dtypes('object').copy()
X_VAL_category = X_VAL.select_dtypes('object').copy()
X_TEST_category = x_test.select_dtypes('object').copy()

enc = OneHotEncoder(sparse_output = False).fit(X_TRAIN_category)

X_TRAIN_OH = enc.transform(X_TRAIN_category)
X_VAL_OH = enc.transform(X_VAL_category)
X_TEST_OH = enc.transform(X_TEST_category)

# 스케일링 : 수치형 컬럼만 별도 저장

X_TRAIN_conti = X_TRAIN.select_dtypes(exclude = 'object').copy()
X_VAL_conti = X_VAL.select_dtypes(exclude = 'object').copy()
X_TEST_conti = x_test.select_dtypes(exclude = 'object').copy()

# 입력 데이터셋 준비

import numpy as np

X_TRAIN = np.concatenate([X_TRAIN_OH, X_TRAIN_conti], axis = 1)
X_VAL = np.concatenate([X_VAL_OH, X_VAL_conti], axis = 1)

y_TRAIN = y_TRAIN.map({'No':0, 'Yes':1})
y_VAL = y_VAL.map({'No':0, 'Yes':1})

y_TRAIN = y_TRAIN.values.ravel()
y_VAL = y_VAL.values.ravel()

# 모델 학습

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

rf = RandomForestClassifier(n_estimators = 500,
                            max_depth = 3,
                            min_samples_leaf = 10,
                            max_features = 'sqrt',
                            random_state = 2022)
model_rf = rf.fit(X_TRAIN, y_TRAIN)

xgb = XGBClassifier(max_depth = 8,
                    n_estimators = 500,
                    nthread = 4,
                    min_child_weight = 20,
                    gamma = 0.5,
                    objective = 'binary:logistic',
                    use_label_encoder = False,
                    random_state = 2022)
model_xgb = xgb.fit(X_TRAIN, y_TRAIN)

# 성능 검정

from sklearn.metrics import f1_score

pred_rf = model_rf.predict(X_VAL)
pred_xgb = model_xgb.predict(X_VAL)

f1_rf = f1_score(y_VAL, pred_rf)
print(f1_rf)

f1_xgb = f1_score(y_VAL, pred_xgb)
print(f1_xgb)

# 결과 제출

X_TEST = np.concatenate([X_TEST_OH, X_TEST_conti], axis = 1)
y_pred = model_xgb.predict(X_TEST)

obj = {'pred':y_pred}
result = pd.DataFrame(obj)
result['pred'] = result['pred'].map({0:'Satisfied', 1:'Not Satisfied'})

result.to_csv('result.csv', index = False)

   Gender      Ages Customer_Type     Class  Inflight_wifi_service  \
0  Female  [10, 20)             L  Eco Plus                      1   
1  Female  [10, 20)             L       Eco                      2   
2  Female  [10, 20)            NL       Eco                      2   
3    Male  [10, 20)            NL       Eco                      5   
4    Male  [10, 20)             L  Business                      3   

   Departure_Arrival_time_convenient  Ease_of_Online_booking  Gate_location  \
0                                  3                       1              3   
1                                  4                       2              4   
2                                  2                       1              4   
3                                  3                       5              3   
4                                  3                       3              3   

   Food_and_drink  Online_boarding  Seat_comfort  Inflight_entertainment  \
0               1           

In [57]:
# 기출복원 7회 2유형

# 데이터 불러오기

import pandas as pd

train = pd.read_csv('Flight_train.csv')
test = pd.read_csv('Flight_test.csv')

print(train.head())
y_train = train['price'].copy()
x_train = train.drop(columns = 'price').copy()

x_test = test.copy()

# 정보 출력

# 불필요한 컬럼 제거

x_train = x_train.drop('flight', axis = 1)
x_test = x_test.drop('flight', axis = 1)

# 결측치 제거

print(x_train.isna().sum())
print(x_test.isna().sum())

# 카테고리형 컬럼 전처리

# 수치형 컬럼 전처리

# 데이터 분할

from sklearn.model_selection import train_test_split

X_TRAIN, X_VAL, y_TRAIN, y_VAL = train_test_split(x_train, y_train, random_state = 1234, test_size = 0.2)

# 인코딩

from sklearn.preprocessing import OneHotEncoder

X_TRAIN_category = X_TRAIN.select_dtypes('object').copy()
X_VAL_category = X_VAL.select_dtypes('object').copy()
X_TEST_category = x_test.select_dtypes('object').copy()

enc = OneHotEncoder(sparse_output = False).fit(X_TRAIN_category)

X_TRAIN_OH = enc.transform(X_TRAIN_category)
X_VAL_OH = enc.transform(X_VAL_category)
X_TEST_OH = enc.transform(X_TEST_category)

# 스케일링 : 수치형 컬럼만 별도 저장

X_TRAIN_conti = X_TRAIN.select_dtypes(exclude = 'object').copy()
X_VAL_conti = X_VAL.select_dtypes(exclude = 'object').copy()
X_TEST_conti = x_test.select_dtypes(exclude = 'object').copy()


# 입력 데이터셋 준비

import numpy as np

X_TRAIN = np.concatenate([X_TRAIN_OH, X_TRAIN_conti], axis = 1)
X_VAL = np.concatenate([X_VAL_OH, X_VAL_conti], axis = 1)

# 모델 학습

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

rf = RandomForestRegressor(n_estimators = 500,
                            max_depth = 3,
                            min_samples_leaf = 10,
                            max_features = 'sqrt',
                            random_state = 2022)
model_rf = rf.fit(X_TRAIN, y_TRAIN)

xgb = XGBRegressor(max_depth = 8,
                    n_estimators = 500,
                    nthread = 4,
                    min_child_weight = 20,
                    gamma = 0.5,
                    objective='reg:squarederror',
                    use_label_encoder = False,
                    random_state = 2022)
model_xgb = xgb.fit(X_TRAIN, y_TRAIN)

# 성능 검정

from sklearn.metrics import mean_squared_error

pred_rf = model_rf.predict(X_VAL)
pred_xgb = model_xgb.predict(X_VAL)

rmse_rf = mean_squared_error(y_VAL, pred_rf)
print(rmse_rf)

rmse_xgb = mean_squared_error(y_VAL, pred_xgb)
print(rmse_xgb)

# 결과 제출

X_TEST = np.concatenate([X_TEST_OH, X_TEST_conti], axis = 1)
y_pred = model_xgb.predict(X_TEST)

obj = {'pred':y_pred}
result = pd.DataFrame(obj)

result.to_csv('result.csv', index = False)

     airline   flight source_city departure_time stops arrival_time  \
0    Vistara   UK-874   Hyderabad        Morning   one    Afternoon   
1  Air_India   AI-687      Mumbai      Afternoon  zero      Evening   
2    Vistara   UK-830   Hyderabad        Morning   one      Morning   
3   GO_FIRST   G8-803   Bangalore        Evening   one   Late_Night   
4     Indigo  6E-5338      Mumbai        Morning   one    Afternoon   

  destination_city     class  duration  days_left  price  
0            Delhi   Economy      5.58         10  10018  
1            Delhi  Business      2.25         12  30080  
2          Kolkata   Economy     23.25         19   5960  
3           Mumbai   Economy      7.17         37   4354  
4          Kolkata   Economy      7.33          8  10406  
airline             0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
dtype: int64
airline 

In [15]:
# 기출복원 7회 2유형 다시 풀기

# 데이터 불러오기

import pandas as pd

train = pd.read_csv('Flight_train.csv')
test = pd.read_csv('Flight_test.csv')

y_train = train['price'].copy()
x_train = train.drop(columns = 'price').copy()

x_test = test.copy()

# 정보 출력

print(x_train.head())
print(x_test.head())
print(y_train.head())

print(x_train.info())
print(x_test.info())
print(y_train.info())

print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

# 불필요한 컬럼 제거

x_train = x_train.drop('flight', axis = 1)
x_test = x_test.drop('flight', axis = 1)

# 결측치 제거, 대체

print(x_train.isna().sum())
print(x_test.isna().sum())

# 카테고리형 컬럼 전처리

## 두 데이터 다른 거 없음
print(x_train.select_dtypes('object').nunique())
print(x_test.select_dtypes('object').nunique())

# 수치형 컬럼 전처리
## 거의 다 object라서 괜찮은건가

# 데이터 분할

from sklearn.model_selection import train_test_split

X_TRAIN, X_VAL, y_TRAIN, y_VAL = train_test_split(x_train, y_train, random_state = 1234, test_size = 0.2)

# 인코딩
## handle_unknown = 'ignore' 옵션 : train에 없는데 test에 존재

from sklearn.preprocessing import OneHotEncoder

X_TRAIN_category = X_TRAIN.select_dtypes('object').copy()
X_VAL_category = X_VAL.select_dtypes('object').copy()
X_TEST_category = x_test.select_dtypes('object').copy()

enc = OneHotEncoder(sparse_output = False).fit(X_TRAIN_category)

X_TRAIN_OH = enc.transform(X_TRAIN_category)
X_VAL_OH = enc.transform(X_VAL_category)
X_TEST_OH = enc.transform(X_TEST_category)

# 스케일링 : Regressor는 스케일링 필요없음

## 수치형만 저장

X_TRAIN_conti = X_TRAIN.select_dtypes(exclude='object').copy()
X_VAL_conti = X_VAL.select_dtypes(exclude='object').copy()
X_TEST_conti = x_test.select_dtypes(exclude='object').copy()

# 입력 데이터셋 준비

import numpy as np

X_TRAIN = np.concatenate([X_TRAIN_OH, X_TRAIN_conti], axis = 1)
X_VAL = np.concatenate([X_VAL_OH, X_VAL_conti], axis = 1)

## y는 이미 1차원이라서 안해줘도 되나 ?

# 모델 학습

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

rf = RandomForestRegressor(n_estimators = 100,
                           max_depth = 3,
                           min_samples_leaf = 10,
                           max_features = 15,
                           random_state = 2022)
model_rf = rf.fit(X_TRAIN, y_TRAIN)

xgb = XGBRegressor(n_estimators = 100,
                   nthread = 5,
                   min_child_weight = 20,
                   gamma = 0.5,
                   objective = 'reg:squarederror',
                   use_label_encoder = False,
                   random_state = 2024)
model_xgb = xgb.fit(X_TRAIN, y_TRAIN)

# 성능 검정

## 제일 낮은거 : rmse

from sklearn.metrics import mean_squared_error

pred_rf = model_rf.predict(X_VAL)
rmse_rf = mean_squared_error(y_VAL, pred_rf, squared = False)
print(rmse_rf)

pred_xgb = model_xgb.predict(X_VAL)
rmse_xgb = mean_squared_error(y_VAL, pred_xgb, squared=False)
print(rmse_xgb)

# 결과 제출

X_TEST = np.concatenate([X_TEST_OH, X_TEST_conti], axis = 1)
y_pred = model_xgb.predict(X_TEST)

obj = {'pred':y_pred}
result = pd.DataFrame(obj)

result.to_csv('result.csv', index = False)


     airline   flight source_city departure_time stops arrival_time  \
0    Vistara   UK-874   Hyderabad        Morning   one    Afternoon   
1  Air_India   AI-687      Mumbai      Afternoon  zero      Evening   
2    Vistara   UK-830   Hyderabad        Morning   one      Morning   
3   GO_FIRST   G8-803   Bangalore        Evening   one   Late_Night   
4     Indigo  6E-5338      Mumbai        Morning   one    Afternoon   

  destination_city     class  duration  days_left  
0            Delhi   Economy      5.58         10  
1            Delhi  Business      2.25         12  
2          Kolkata   Economy     23.25         19  
3           Mumbai   Economy      7.17         37  
4          Kolkata   Economy      7.33          8  
     airline   flight source_city departure_time stops arrival_time  \
0     Indigo  6E-6219     Chennai        Morning   one    Afternoon   
1  Air_India   AI-503   Bangalore        Evening   one      Morning   
2    Vistara   UK-830   Hyderabad        Morning